In [22]:
import pandas as pd
import numpy as np
import os
import datetime

In [85]:
dir_path = "C:/Users/Admin/Desktop/Columbia/E4524 Capstone/Final_Data/Sentiment_Data/"
sentiment_list = []
sentiment_files = []
for file in os.listdir(dir_path):
    sentiment_list.append(dir_path + file)
    sentiment_files.append(file[:-4])
sentiment_list, sentiment_files

(['C:/Users/Admin/Desktop/Columbia/E4524 Capstone/Final_Data/Sentiment_Data/1.gt_bitcoin.csv',
  'C:/Users/Admin/Desktop/Columbia/E4524 Capstone/Final_Data/Sentiment_Data/10.news_senti.csv',
  'C:/Users/Admin/Desktop/Columbia/E4524 Capstone/Final_Data/Sentiment_Data/11.fear&greed.csv',
  'C:/Users/Admin/Desktop/Columbia/E4524 Capstone/Final_Data/Sentiment_Data/12.tweets_pos.csv',
  'C:/Users/Admin/Desktop/Columbia/E4524 Capstone/Final_Data/Sentiment_Data/13.tweets_neg.csv',
  'C:/Users/Admin/Desktop/Columbia/E4524 Capstone/Final_Data/Sentiment_Data/14.twitter_btccrash_pos.csv',
  'C:/Users/Admin/Desktop/Columbia/E4524 Capstone/Final_Data/Sentiment_Data/15.twitter_btccrash_neg.csv',
  'C:/Users/Admin/Desktop/Columbia/E4524 Capstone/Final_Data/Sentiment_Data/2.gt_Buy_Bitcoin.csv',
  'C:/Users/Admin/Desktop/Columbia/E4524 Capstone/Final_Data/Sentiment_Data/3.gt_Sell_Bitcoin.csv',
  'C:/Users/Admin/Desktop/Columbia/E4524 Capstone/Final_Data/Sentiment_Data/4.gt_ethereum.csv',
  'C:/Users/Ad

In [86]:
dir_path = "C:/Users/Admin/Desktop/Columbia/E4524 Capstone/Final_Data/Crypto_Data/"
crypto_list = []
crypto_files = []
for file in os.listdir(dir_path):
    crypto_list.append(dir_path + file)
    crypto_files.append(file[:-4])
crypto_list,crypto_files

(['C:/Users/Admin/Desktop/Columbia/E4524 Capstone/Final_Data/Crypto_Data/bitcoin.csv',
  'C:/Users/Admin/Desktop/Columbia/E4524 Capstone/Final_Data/Crypto_Data/ethereum.csv',
  'C:/Users/Admin/Desktop/Columbia/E4524 Capstone/Final_Data/Crypto_Data/SP_BDM.csv',
  'C:/Users/Admin/Desktop/Columbia/E4524 Capstone/Final_Data/Crypto_Data/SP_BDM_exlarge.csv'],
 ['bitcoin', 'ethereum', 'SP_BDM', 'SP_BDM_exlarge'])

In [87]:
sentiment_list[0][73:-4], crypto_list[0][70:-4]

('1.gt_bitcoin', 'bitcoin')

In [98]:
def granger_matrix(c_column, s_column, direction, maxlag, method = 'ssr_chi2test'):
    from statsmodels.tsa.stattools import grangercausalitytests
    index = [i+1 for i in range(maxlag)]
    for crypto_file in crypto_list:
        df_crypto = pd.read_csv(crypto_file)
        df_crypto = df_crypto[['date',c_column]]
        df_result = pd.DataFrame(np.zeros((maxlag, len(sentiment_list))), columns = sentiment_files, index = index)
        for sentiment_file in sentiment_list:
            df_sentiment = pd.read_csv(sentiment_file)
            df_sentiment = df_sentiment[['date',s_column]]
            df_test = df_sentiment.merge(df_crypto, on = 'date', how = 'left')
            df_test.replace([np.inf, -np.inf], np.nan, inplace=True)
            df_test.dropna(inplace = True) # columns = ['date',s_column, c_column]
            if direction == 's_predict_c':
                test_result = grangercausalitytests(df_test.iloc[:,[2,1]], maxlag = maxlag, verbose = False)
            elif direction == 'c_predict_s':
                test_result = grangercausalitytests(df_test.iloc[:,[1,2]], maxlag = maxlag, verbose = False)
            p_values = [round(test_result[i+1][0][method][1],4) for i in range(maxlag)]
            df_result[sentiment_file[73:-4]] = p_values
        print('Granger Causality Test: sentiment indices vs.',crypto_file[70:-4])
        display(df_result)

### 1. Level

In [99]:
# index -> price
granger_matrix(c_column = 'price', s_column = 'index', direction = 's_predict_c', maxlag = 5)

Granger Causality Test: sentiment indices vs. bitcoin


,1.gt_bitcoin,10.news_senti,11.fear&greed,12.tweets_pos,13.tweets_neg,14.twitter_btccrash_pos,15.twitter_btccrash_neg,2.gt_Buy_Bitcoin,3.gt_Sell_Bitcoin,4.gt_ethereum,5.gt_Ukraine_war,6.gt_covid,7.s&p_twitter,8.CBDC_uncertainty,9.CBDC_atten
1,0.7437,0.2953,0.0181,0.2265,0.9683,0.2127,0.2040,0.0005,0.0132,0.4644,0.4878,0.0081,0.1668,0.2911,0.0407
2,0.9515,0.5137,0.0489,0.1761,0.9808,0.4429,0.4578,0.0000,0.0101,0.0064,0.3637,0.0163,0.3742,0.4149,0.0830
3,0.8549,0.4808,0.0850,0.1411,0.9971,0.6404,0.1557,0.0001,0.0256,0.0157,0.5212,0.0299,0.4767,0.6079,0.1434
4,0.2332,0.6718,0.1417,0.1553,0.9856,0.7598,0.2515,0.0003,0.0265,0.0002,0.3242,0.0423,0.5551,0.2641,0.1820
5,0.0800,0.7417,0.3920,0.2176,0.8753,0.8740,0.3504,0.0000,0.0001,0.0008,0.0255,0.0267,0.3108,0.2735,0.2759


Granger Causality Test: sentiment indices vs. ethereum


,1.gt_bitcoin,10.news_senti,11.fear&greed,12.tweets_pos,13.tweets_neg,14.twitter_btccrash_pos,15.twitter_btccrash_neg,2.gt_Buy_Bitcoin,3.gt_Sell_Bitcoin,4.gt_ethereum,5.gt_Ukraine_war,6.gt_covid,7.s&p_twitter,8.CBDC_uncertainty,9.CBDC_atten
1,0.7410,0.7484,0.0186,0.3530,0.8629,0.1895,0.1443,0.0167,0.0535,0.0004,0.4509,0.0712,0.0027,0.0257,0.0004
2,0.6019,0.3190,0.0289,0.3477,0.9187,0.4007,0.3032,0.0043,0.0207,0.0000,0.4254,0.0312,0.0031,0.0681,0.0019
3,0.3174,0.0927,0.0345,0.3514,0.4377,0.5479,0.2954,0.0147,0.0538,0.0000,0.2019,0.0791,0.0090,0.1181,0.0042
4,0.0000,0.1837,0.1371,0.2274,0.6475,0.6987,0.4438,0.0269,0.0007,0.0000,0.2453,0.0567,0.0166,0.0682,0.0085
5,0.0000,0.2927,0.2772,0.3294,0.6057,0.8506,0.5984,0.0350,0.0005,0.0000,0.1667,0.0674,0.0268,0.0320,0.0070


Granger Causality Test: sentiment indices vs. SP_BDM


,1.gt_bitcoin,10.news_senti,11.fear&greed,12.tweets_pos,13.tweets_neg,14.twitter_btccrash_pos,15.twitter_btccrash_neg,2.gt_Buy_Bitcoin,3.gt_Sell_Bitcoin,4.gt_ethereum,5.gt_Ukraine_war,6.gt_covid,7.s&p_twitter,8.CBDC_uncertainty,9.CBDC_atten
1,0.8019,0.3345,0.0008,0.0697,0.0537,0.1603,0.3109,0.0005,0.0139,0.3757,0.2377,0.0031,0.1162,0.5114,0.2255
2,0.9698,0.6063,0.0036,0.1810,0.1395,0.3750,0.5586,0.0001,0.0044,0.0000,0.3337,0.0018,0.1318,0.6682,0.2515
3,0.7189,0.7961,0.0304,0.2304,0.3179,0.5677,0.5070,0.0004,0.0206,0.0005,0.1339,0.0072,0.1966,0.8686,0.5980
4,0.0016,0.3013,0.0657,0.0323,0.2222,0.6861,0.6237,0.0018,0.0008,0.0005,0.1237,0.0022,0.2805,0.8665,0.7833
5,0.0020,0.4681,0.2279,0.0666,0.3148,0.8217,0.7239,0.0002,0.0001,0.0005,0.0277,0.0001,0.3207,0.8802,0.8604


Granger Causality Test: sentiment indices vs. SP_BDM_exlarge


,1.gt_bitcoin,10.news_senti,11.fear&greed,12.tweets_pos,13.tweets_neg,14.twitter_btccrash_pos,15.twitter_btccrash_neg,2.gt_Buy_Bitcoin,3.gt_Sell_Bitcoin,4.gt_ethereum,5.gt_Ukraine_war,6.gt_covid,7.s&p_twitter,8.CBDC_uncertainty,9.CBDC_atten
1,0.9971,0.9621,0.0038,0.0149,0.0578,0.2813,0.1663,0.0099,0.0876,0.0169,0.2149,0.0009,0.0779,0.9649,0.6461
2,0.9266,0.9846,0.0082,0.0379,0.0827,0.5487,0.2801,0.0154,0.0603,0.0000,0.3141,0.0002,0.0372,0.5486,0.6927
3,0.9377,0.9669,0.0340,0.0858,0.1763,0.6955,0.4092,0.0471,0.0579,0.0001,0.1770,0.0013,0.0606,0.5891,0.5858
4,0.0000,0.4543,0.0567,0.0020,0.2582,0.8228,0.5914,0.0528,0.0000,0.0002,0.2278,0.0005,0.0843,0.5127,0.5131
5,0.0000,0.4777,0.1810,0.0044,0.4994,0.9162,0.7390,0.0629,0.0000,0.0000,0.3235,0.0010,0.0723,0.5618,0.6005


In [100]:
# price -> index
granger_matrix(c_column = 'price', s_column = 'index', direction = 'c_predict_s', maxlag = 5)

Granger Causality Test: sentiment indices vs. bitcoin


,1.gt_bitcoin,10.news_senti,11.fear&greed,12.tweets_pos,13.tweets_neg,14.twitter_btccrash_pos,15.twitter_btccrash_neg,2.gt_Buy_Bitcoin,3.gt_Sell_Bitcoin,4.gt_ethereum,5.gt_Ukraine_war,6.gt_covid,7.s&p_twitter,8.CBDC_uncertainty,9.CBDC_atten
1,0.0173,0.0666,0.2097,0.0,0.0000,0.0107,0.4112,0.1065,0.9137,0.0004,0.2985,0.0164,0.2102,0.0005,0.0
2,0.0000,0.0820,0.0000,0.0,0.0000,0.0524,0.6930,0.0091,0.0001,0.0000,0.1242,0.1607,0.4603,0.0000,0.0
3,0.0000,0.0949,0.0000,0.0,0.0003,0.1377,0.8587,0.0406,0.0016,0.0000,0.2393,0.3354,0.6537,0.0000,0.0
4,0.0000,0.1469,0.0000,0.0,0.0007,0.2147,0.8872,0.1311,0.0102,0.0000,0.1058,0.1451,0.7170,0.0000,0.0
5,0.0000,0.2305,0.0000,0.0,0.0007,0.3103,0.6977,0.0195,0.0001,0.0000,0.1786,0.2915,0.8238,0.0000,0.0


Granger Causality Test: sentiment indices vs. ethereum


,1.gt_bitcoin,10.news_senti,11.fear&greed,12.tweets_pos,13.tweets_neg,14.twitter_btccrash_pos,15.twitter_btccrash_neg,2.gt_Buy_Bitcoin,3.gt_Sell_Bitcoin,4.gt_ethereum,5.gt_Ukraine_war,6.gt_covid,7.s&p_twitter,8.CBDC_uncertainty,9.CBDC_atten
1,0.0461,0.2315,0.8416,0.0,0.0000,0.0001,0.0514,0.1871,0.6292,0.5189,0.1402,0.1058,0.9810,0.0561,0.1323
2,0.0002,0.0462,0.0000,0.0,0.0001,0.0007,0.1893,0.3452,0.1130,0.0000,0.0324,0.3651,0.6097,0.0000,0.0000
3,0.0000,0.0747,0.0000,0.0,0.0008,0.0037,0.3138,0.4805,0.0490,0.0003,0.0899,0.4063,0.8185,0.0000,0.0000
4,0.0000,0.1801,0.0000,0.0,0.0043,0.0030,0.2912,0.5502,0.1573,0.0020,0.0391,0.0397,0.7964,0.0000,0.0000
5,0.0000,0.1587,0.0000,0.0,0.0051,0.0121,0.4381,0.1205,0.0003,0.0061,0.0898,0.0202,0.9082,0.0000,0.0000


Granger Causality Test: sentiment indices vs. SP_BDM


,1.gt_bitcoin,10.news_senti,11.fear&greed,12.tweets_pos,13.tweets_neg,14.twitter_btccrash_pos,15.twitter_btccrash_neg,2.gt_Buy_Bitcoin,3.gt_Sell_Bitcoin,4.gt_ethereum,5.gt_Ukraine_war,6.gt_covid,7.s&p_twitter,8.CBDC_uncertainty,9.CBDC_atten
1,0.0229,0.1233,0.6328,0.0,0.0000,0.0035,0.2736,0.0989,0.9411,0.0361,0.2345,0.0074,0.4618,0.0018,0.0
2,0.0000,0.1759,0.0000,0.0,0.0001,0.0216,0.5906,0.0188,0.0002,0.0000,0.1722,0.2230,0.8124,0.0000,0.0
3,0.0000,0.1770,0.0000,0.0,0.0002,0.0341,0.7164,0.0218,0.0000,0.0000,0.3246,0.2896,0.8603,0.0000,0.0
4,0.0000,0.2413,0.0000,0.0,0.0004,0.0695,0.6764,0.0523,0.0001,0.0000,0.1024,0.3126,0.9345,0.0000,0.0
5,0.0000,0.1488,0.0000,0.0,0.0004,0.0589,0.8154,0.0371,0.0000,0.0000,0.1670,0.4416,0.9750,0.0000,0.0


Granger Causality Test: sentiment indices vs. SP_BDM_exlarge


,1.gt_bitcoin,10.news_senti,11.fear&greed,12.tweets_pos,13.tweets_neg,14.twitter_btccrash_pos,15.twitter_btccrash_neg,2.gt_Buy_Bitcoin,3.gt_Sell_Bitcoin,4.gt_ethereum,5.gt_Ukraine_war,6.gt_covid,7.s&p_twitter,8.CBDC_uncertainty,9.CBDC_atten
1,0.0361,0.3295,0.92,0.0,0.0000,0.0190,0.4662,0.0929,0.9647,0.2676,0.2080,0.0001,0.8126,0.0035,0.0
2,0.0000,0.3376,0.00,0.0,0.0013,0.0861,0.7926,0.0244,0.0008,0.0000,0.4189,0.4432,0.4989,0.0000,0.0
3,0.0000,0.5077,0.00,0.0,0.0096,0.0563,0.7805,0.0053,0.0000,0.0000,0.6336,0.4012,0.6395,0.0000,0.0
4,0.0000,0.6017,0.00,0.0,0.0130,0.1079,0.7098,0.0058,0.0000,0.0000,0.2975,0.5241,0.7762,0.0000,0.0
5,0.0000,0.0624,0.00,0.0,0.0039,0.0948,0.7903,0.0102,0.0000,0.0000,0.4232,0.5507,0.8613,0.0000,0.0


### 2. Return

In [102]:
# index return -> price return
granger_matrix(c_column = 'return', s_column = 'return', direction = 's_predict_c', maxlag = 5)

Granger Causality Test: sentiment indices vs. bitcoin


,1.gt_bitcoin,10.news_senti,11.fear&greed,12.tweets_pos,13.tweets_neg,14.twitter_btccrash_pos,15.twitter_btccrash_neg,2.gt_Buy_Bitcoin,3.gt_Sell_Bitcoin,4.gt_ethereum,5.gt_Ukraine_war,6.gt_covid,7.s&p_twitter,8.CBDC_uncertainty,9.CBDC_atten
1,0.6030,0.0590,0.6912,0.6317,0.9186,0.9054,0.7357,0.0224,0.7457,0.0341,0.4132,0.1016,0.4889,0.3228,0.2760
2,0.7525,0.0921,0.3399,0.6313,0.7057,0.7439,0.8654,0.0269,0.4478,0.0007,0.2681,0.5705,0.7284,0.6151,0.5311
3,0.8236,0.0889,0.1723,0.8196,0.8599,0.1837,0.7622,0.0637,0.6548,0.0014,0.3610,0.7214,0.8890,0.5229,0.7045
4,0.8401,0.1861,0.5112,0.8310,0.9095,0.3053,0.8776,0.1383,0.4480,0.0032,0.4155,0.4467,0.8322,0.6343,0.8103
5,0.8552,0.2861,0.5989,0.8066,0.7997,0.0686,0.9417,0.1644,0.5925,0.0077,0.2679,0.3331,0.7976,0.7789,0.9131


Granger Causality Test: sentiment indices vs. ethereum


,1.gt_bitcoin,10.news_senti,11.fear&greed,12.tweets_pos,13.tweets_neg,14.twitter_btccrash_pos,15.twitter_btccrash_neg,2.gt_Buy_Bitcoin,3.gt_Sell_Bitcoin,4.gt_ethereum,5.gt_Ukraine_war,6.gt_covid,7.s&p_twitter,8.CBDC_uncertainty,9.CBDC_atten
1,0.8379,0.0251,0.6700,0.8968,0.8558,0.9757,0.4124,0.0245,0.8608,0.0159,0.6308,0.0343,0.7645,0.7370,0.2217
2,0.7208,0.0443,0.1324,0.7512,0.7681,0.8791,0.6545,0.0741,0.8804,0.0002,0.2117,0.3207,0.6615,0.7791,0.3943
3,0.2265,0.0289,0.1907,0.8975,0.8687,0.6550,0.8411,0.1492,0.9687,0.0003,0.3989,0.5416,0.8302,0.3437,0.5356
4,0.2405,0.0675,0.3784,0.9733,0.8981,0.8080,0.8549,0.2742,0.6579,0.0011,0.5718,0.1897,0.8398,0.3558,0.6989
5,0.1730,0.0932,0.5085,0.9312,0.9538,0.5744,0.9321,0.3905,0.7827,0.0015,0.6524,0.0359,0.8498,0.5007,0.8209


Granger Causality Test: sentiment indices vs. SP_BDM


,1.gt_bitcoin,10.news_senti,11.fear&greed,12.tweets_pos,13.tweets_neg,14.twitter_btccrash_pos,15.twitter_btccrash_neg,2.gt_Buy_Bitcoin,3.gt_Sell_Bitcoin,4.gt_ethereum,5.gt_Ukraine_war,6.gt_covid,7.s&p_twitter,8.CBDC_uncertainty,9.CBDC_atten
1,0.7738,0.1451,0.6301,0.3267,0.3175,0.5662,0.2467,0.0292,0.7980,0.0177,0.4740,0.0519,0.8328,0.4593,0.2020
2,0.8508,0.1783,0.4553,0.3838,0.3238,0.8174,0.5024,0.0527,0.7968,0.0006,0.3538,0.1301,0.7353,0.7229,0.4204
3,0.9212,0.0734,0.8858,0.4068,0.1980,0.2358,0.3763,0.1359,0.9291,0.0016,0.1382,0.5946,0.8591,0.8564,0.5767
4,0.5742,0.1373,0.9463,0.5341,0.3368,0.2975,0.4593,0.2172,0.6243,0.0016,0.1788,0.3728,0.8177,0.9256,0.7383
5,0.1050,0.2206,0.8941,0.2684,0.3808,0.4078,0.4610,0.3209,0.6774,0.0020,0.2408,0.3853,0.7704,0.9525,0.8541


Granger Causality Test: sentiment indices vs. SP_BDM_exlarge


,1.gt_bitcoin,10.news_senti,11.fear&greed,12.tweets_pos,13.tweets_neg,14.twitter_btccrash_pos,15.twitter_btccrash_neg,2.gt_Buy_Bitcoin,3.gt_Sell_Bitcoin,4.gt_ethereum,5.gt_Ukraine_war,6.gt_covid,7.s&p_twitter,8.CBDC_uncertainty,9.CBDC_atten
1,0.1197,0.6329,0.3299,0.1736,0.5151,0.1642,0.1526,0.9197,0.2555,0.1929,0.3395,0.0440,0.4000,0.5768,0.2365
2,0.2314,0.2261,0.2672,0.3313,0.2241,0.3902,0.3027,0.4610,0.3032,0.0032,0.1520,0.1184,0.1878,0.7195,0.3408
3,0.0175,0.2973,0.6367,0.1129,0.2361,0.2149,0.2376,0.1876,0.2209,0.0103,0.0164,0.6196,0.2729,0.7803,0.3887
4,0.0165,0.4037,0.7064,0.1047,0.3302,0.3403,0.2837,0.3095,0.1191,0.0223,0.0584,0.2729,0.1912,0.8892,0.5623
5,0.0005,0.5131,0.8000,0.0546,0.3898,0.4867,0.2842,0.3403,0.0720,0.0291,0.1095,0.2900,0.2430,0.9433,0.6748


In [103]:
# price return -> index return
granger_matrix(c_column = 'return', s_column = 'return', direction = 'c_predict_s', maxlag = 5)

Granger Causality Test: sentiment indices vs. bitcoin


,1.gt_bitcoin,10.news_senti,11.fear&greed,12.tweets_pos,13.tweets_neg,14.twitter_btccrash_pos,15.twitter_btccrash_neg,2.gt_Buy_Bitcoin,3.gt_Sell_Bitcoin,4.gt_ethereum,5.gt_Ukraine_war,6.gt_covid,7.s&p_twitter,8.CBDC_uncertainty,9.CBDC_atten
1,0.1459,0.0148,0.0,0.3090,0.6447,0.4257,0.6439,0.5874,0.2367,0.9665,0.4302,0.1838,0.1486,0.3204,0.9018
2,0.3701,0.0371,0.0,0.3741,0.0799,0.6805,0.8522,0.8181,0.4601,0.8833,0.3854,0.6360,0.2767,0.4038,0.6254
3,0.5420,0.0742,0.0,0.5047,0.1660,0.7932,0.9232,0.8636,0.5878,0.6397,0.4186,0.4354,0.1698,0.5473,0.1559
4,0.2290,0.1343,0.0,0.3184,0.2883,0.8878,0.8415,0.8146,0.6758,0.5670,0.5341,0.5867,0.2814,0.7705,0.2218
5,0.3075,0.2174,0.0,0.3746,0.1398,0.8690,0.8533,0.8724,0.6920,0.4821,0.6172,0.4071,0.2876,0.7941,0.2822


Granger Causality Test: sentiment indices vs. ethereum


,1.gt_bitcoin,10.news_senti,11.fear&greed,12.tweets_pos,13.tweets_neg,14.twitter_btccrash_pos,15.twitter_btccrash_neg,2.gt_Buy_Bitcoin,3.gt_Sell_Bitcoin,4.gt_ethereum,5.gt_Ukraine_war,6.gt_covid,7.s&p_twitter,8.CBDC_uncertainty,9.CBDC_atten
1,0.0981,0.0309,0.0,0.1450,0.7732,0.3296,0.8894,0.8584,0.9579,0.5298,0.3548,0.1814,0.3115,0.5835,0.7741
2,0.1066,0.0669,0.0,0.1449,0.1946,0.5683,0.9395,0.8945,0.9979,0.8157,0.3854,0.4757,0.4697,0.6287,0.5835
3,0.2718,0.1403,0.0,0.2561,0.1959,0.5292,0.9742,0.9691,0.9993,0.5896,0.3370,0.4957,0.1145,0.4738,0.5153
4,0.2158,0.1729,0.0,0.1897,0.3824,0.5876,0.9865,0.7218,0.9224,0.4811,0.2747,0.6914,0.0734,0.6332,0.6583
5,0.3987,0.2706,0.0,0.2406,0.1835,0.5103,0.9853,0.8491,0.9623,0.3618,0.2416,0.5068,0.0708,0.5224,0.4997


Granger Causality Test: sentiment indices vs. SP_BDM


,1.gt_bitcoin,10.news_senti,11.fear&greed,12.tweets_pos,13.tweets_neg,14.twitter_btccrash_pos,15.twitter_btccrash_neg,2.gt_Buy_Bitcoin,3.gt_Sell_Bitcoin,4.gt_ethereum,5.gt_Ukraine_war,6.gt_covid,7.s&p_twitter,8.CBDC_uncertainty,9.CBDC_atten
1,0.1176,0.3597,0.0,0.4078,0.6213,0.5030,0.5531,0.2740,0.6854,0.6044,0.9498,0.1109,0.6000,0.3828,0.8635
2,0.1072,0.4274,0.0,0.7547,0.1174,0.6391,0.8470,0.3187,0.8706,0.7939,0.8253,0.5655,0.1407,0.0891,0.9634
3,0.2434,0.5191,0.0,0.8381,0.1555,0.8074,0.9126,0.3852,0.8779,0.4129,0.6416,0.3566,0.0711,0.1737,0.5251
4,0.1949,0.6652,0.0,0.3696,0.1624,0.8986,0.9689,0.3836,0.7647,0.0718,0.0988,0.3614,0.1620,0.2565,0.5499
5,0.2298,0.7848,0.0,0.4862,0.0201,0.7659,0.9891,0.4989,0.8565,0.0853,0.0172,0.1502,0.2306,0.3068,0.5928


Granger Causality Test: sentiment indices vs. SP_BDM_exlarge


,1.gt_bitcoin,10.news_senti,11.fear&greed,12.tweets_pos,13.tweets_neg,14.twitter_btccrash_pos,15.twitter_btccrash_neg,2.gt_Buy_Bitcoin,3.gt_Sell_Bitcoin,4.gt_ethereum,5.gt_Ukraine_war,6.gt_covid,7.s&p_twitter,8.CBDC_uncertainty,9.CBDC_atten
1,0.0892,0.7673,0.0,0.1596,0.9974,0.6511,0.4971,0.1127,0.7248,0.7708,0.7250,0.1447,0.3086,0.7237,0.9979
2,0.0630,0.8829,0.0,0.6224,0.2346,0.7267,0.6527,0.0893,0.8962,0.9311,0.4064,0.6386,0.1980,0.1766,0.9827
3,0.1720,0.9552,0.0,0.5817,0.3695,0.5828,0.8013,0.1243,0.7730,0.4241,0.2826,0.3209,0.0678,0.2915,0.7194
4,0.0951,0.9429,0.0,0.4648,0.3685,0.7099,0.8267,0.1406,0.7621,0.2622,0.0325,0.1275,0.1628,0.3776,0.7907
5,0.2389,0.9644,0.0,0.7154,0.0302,0.4847,0.8689,0.3297,0.9175,0.1988,0.0006,0.0428,0.2108,0.4373,0.7462


### 3. Changes in Volatility

In [104]:
# index changes in volatility -> price changes in volatility
granger_matrix(c_column = 'change_in_volatility', s_column = 'change_in_volatility', direction = 's_predict_c', maxlag = 5)

Granger Causality Test: sentiment indices vs. bitcoin


,1.gt_bitcoin,10.news_senti,11.fear&greed,12.tweets_pos,13.tweets_neg,14.twitter_btccrash_pos,15.twitter_btccrash_neg,2.gt_Buy_Bitcoin,3.gt_Sell_Bitcoin,4.gt_ethereum,5.gt_Ukraine_war,6.gt_covid,7.s&p_twitter,8.CBDC_uncertainty,9.CBDC_atten
1,0.3503,0.4153,0.4808,0.0540,0.5111,0.8259,0.7404,0.4645,0.3154,0.7668,0.7273,0.6314,0.5827,0.3823,0.2877
2,0.6329,0.7325,0.1853,0.1184,0.8151,0.8798,0.9246,0.7756,0.5430,0.7421,0.6779,0.8144,0.8220,0.5826,0.2547
3,0.6504,0.8887,0.0265,0.2147,0.8031,0.9636,0.9549,0.7818,0.6230,0.5708,0.2617,0.9099,0.8968,0.3735,0.3135
4,0.0880,0.9562,0.0987,0.3257,0.8661,0.9886,0.9855,0.4294,0.2400,0.1097,0.2191,0.8579,0.9489,0.4858,0.0315
5,0.0312,0.9799,0.1633,0.4573,0.9411,0.3575,0.9831,0.1498,0.2650,0.1301,0.2538,0.9149,0.9094,0.6197,0.0147


Granger Causality Test: sentiment indices vs. ethereum


,1.gt_bitcoin,10.news_senti,11.fear&greed,12.tweets_pos,13.tweets_neg,14.twitter_btccrash_pos,15.twitter_btccrash_neg,2.gt_Buy_Bitcoin,3.gt_Sell_Bitcoin,4.gt_ethereum,5.gt_Ukraine_war,6.gt_covid,7.s&p_twitter,8.CBDC_uncertainty,9.CBDC_atten
1,0.1566,0.1493,0.8674,0.1971,0.6923,0.7895,0.5848,0.1507,0.8063,0.9520,0.4462,0.6485,0.4117,0.5899,0.6454
2,0.1392,0.3812,0.5176,0.2979,0.7865,0.9296,0.8340,0.3904,0.7735,0.9656,0.4831,0.9199,0.7400,0.8553,0.9333
3,0.1875,0.5407,0.2963,0.5033,0.8618,0.9839,0.9365,0.3088,0.8418,0.8599,0.4311,0.9100,0.4813,0.6103,0.5731
4,0.0397,0.7068,0.3622,0.6436,0.9250,0.9961,0.9835,0.4080,0.1529,0.7314,0.5996,0.7958,0.5466,0.7467,0.2606
5,0.0120,0.8267,0.3898,0.7524,0.8232,0.9826,0.9123,0.0272,0.0802,0.2535,0.7157,0.6624,0.4227,0.8098,0.2434


Granger Causality Test: sentiment indices vs. SP_BDM


,1.gt_bitcoin,10.news_senti,11.fear&greed,12.tweets_pos,13.tweets_neg,14.twitter_btccrash_pos,15.twitter_btccrash_neg,2.gt_Buy_Bitcoin,3.gt_Sell_Bitcoin,4.gt_ethereum,5.gt_Ukraine_war,6.gt_covid,7.s&p_twitter,8.CBDC_uncertainty,9.CBDC_atten
1,0.4815,0.6633,0.3632,0.0663,0.6993,0.8536,0.5616,0.8827,0.9900,0.5034,0.6702,0.4507,0.1403,0.1544,0.0799
2,0.2668,0.9090,0.0894,0.1696,0.7667,0.9732,0.8519,0.9079,0.6217,0.7802,0.7231,0.6567,0.0581,0.2409,0.1568
3,0.4310,0.9345,0.1468,0.2323,0.9120,0.9515,0.9412,0.7286,0.2277,0.9387,0.4208,0.2469,0.0256,0.4549,0.1934
4,0.3836,0.9786,0.1146,0.2962,0.9815,0.9868,0.9749,0.8795,0.2404,0.9351,0.5405,0.0587,0.0659,0.4022,0.0895
5,0.0718,0.9883,0.1691,0.3340,0.8864,0.9961,0.9928,0.1522,0.0256,0.6404,0.6855,0.0904,0.0221,0.4065,0.0198


Granger Causality Test: sentiment indices vs. SP_BDM_exlarge


,1.gt_bitcoin,10.news_senti,11.fear&greed,12.tweets_pos,13.tweets_neg,14.twitter_btccrash_pos,15.twitter_btccrash_neg,2.gt_Buy_Bitcoin,3.gt_Sell_Bitcoin,4.gt_ethereum,5.gt_Ukraine_war,6.gt_covid,7.s&p_twitter,8.CBDC_uncertainty,9.CBDC_atten
1,0.5125,0.9869,0.3490,0.4970,0.8731,0.5895,0.5538,0.8874,0.8614,0.2210,0.7402,0.2925,0.0436,0.1467,0.2177
2,0.2665,0.9760,0.1084,0.7909,0.3283,0.7037,0.8453,0.9091,0.5244,0.4234,0.8410,0.4221,0.0093,0.2181,0.4947
3,0.4536,0.9929,0.1717,0.5216,0.4731,0.8500,0.9209,0.3743,0.1353,0.6272,0.6056,0.2254,0.0020,0.4245,0.5418
4,0.6409,0.9932,0.0951,0.6054,0.7233,0.8311,0.9244,0.4442,0.2551,0.7164,0.7631,0.0125,0.0091,0.0425,0.7253
5,0.1848,0.9978,0.1314,0.4641,0.5056,0.8628,0.9742,0.0848,0.0009,0.3768,0.8579,0.0233,0.0039,0.0269,0.3496


In [105]:
# index changes in volatility -> price changes in volatility
granger_matrix(c_column = 'change_in_volatility', s_column = 'change_in_volatility', direction = 'c_predict_s', maxlag = 5)

Granger Causality Test: sentiment indices vs. bitcoin


,1.gt_bitcoin,10.news_senti,11.fear&greed,12.tweets_pos,13.tweets_neg,14.twitter_btccrash_pos,15.twitter_btccrash_neg,2.gt_Buy_Bitcoin,3.gt_Sell_Bitcoin,4.gt_ethereum,5.gt_Ukraine_war,6.gt_covid,7.s&p_twitter,8.CBDC_uncertainty,9.CBDC_atten
1,0.7273,0.1448,0.0,0.4930,0.2150,0.7836,0.9922,0.4885,0.2810,0.2025,0.7225,0.3085,0.0154,0.5547,0.0004
2,0.9079,0.2977,0.0,0.0405,0.2708,0.6639,0.8024,0.7853,0.5467,0.4171,0.9282,0.0629,0.0376,0.1879,0.0004
3,0.9811,0.4719,0.0,0.0910,0.4477,0.8448,0.7228,0.9211,0.7502,0.6338,0.9835,0.1174,0.0529,0.1859,0.0012
4,0.9953,0.6407,0.0,0.1122,0.5726,0.9223,0.8531,0.7559,0.6436,0.7656,0.9943,0.0622,0.0941,0.2079,0.0025
5,0.9983,0.7600,0.0,0.1866,0.3041,0.9620,0.9164,0.6831,0.4948,0.8063,0.9981,0.0469,0.0354,0.3198,0.0039


Granger Causality Test: sentiment indices vs. ethereum


,1.gt_bitcoin,10.news_senti,11.fear&greed,12.tweets_pos,13.tweets_neg,14.twitter_btccrash_pos,15.twitter_btccrash_neg,2.gt_Buy_Bitcoin,3.gt_Sell_Bitcoin,4.gt_ethereum,5.gt_Ukraine_war,6.gt_covid,7.s&p_twitter,8.CBDC_uncertainty,9.CBDC_atten
1,0.7649,0.1095,0.0,0.1659,0.0908,0.0246,0.6226,0.5296,0.5585,0.4012,0.4120,0.5535,0.0180,0.9858,0.0851
2,0.9836,0.2769,0.0,0.0055,0.1603,0.0799,0.4786,0.8532,0.4819,0.6181,0.6585,0.8685,0.0314,0.0679,0.0006
3,0.9875,0.4389,0.0,0.0166,0.2181,0.1681,0.5694,0.6746,0.6829,0.6373,0.8436,0.9355,0.0065,0.1431,0.0016
4,0.9926,0.5712,0.0,0.0188,0.1751,0.2731,0.7335,0.7177,0.5101,0.7797,0.9251,0.8748,0.0154,0.2411,0.0023
5,0.8945,0.6636,0.0,0.0375,0.0995,0.3956,0.8485,0.5521,0.2678,0.8723,0.9688,0.1711,0.0057,0.3442,0.0041


Granger Causality Test: sentiment indices vs. SP_BDM


,1.gt_bitcoin,10.news_senti,11.fear&greed,12.tweets_pos,13.tweets_neg,14.twitter_btccrash_pos,15.twitter_btccrash_neg,2.gt_Buy_Bitcoin,3.gt_Sell_Bitcoin,4.gt_ethereum,5.gt_Ukraine_war,6.gt_covid,7.s&p_twitter,8.CBDC_uncertainty,9.CBDC_atten
1,0.9345,0.9257,0.0,0.8727,0.0470,0.9810,0.3630,0.7708,0.9459,0.9126,0.8226,0.9452,0.5015,0.3699,0.0166
2,0.8701,0.9952,0.0,0.4150,0.0617,0.9559,0.5561,0.4836,0.1179,0.8603,0.9492,0.9890,0.6523,0.1579,0.0055
3,0.8889,0.9942,0.0,0.6455,0.1365,0.9930,0.7195,0.5326,0.2270,0.9036,0.9829,0.8810,0.0183,0.1536,0.0128
4,0.9523,0.9992,0.0,0.7482,0.2154,0.9964,0.8145,0.3711,0.2398,0.9627,0.9921,0.4459,0.0039,0.2121,0.0263
5,0.9821,0.9998,0.0,0.8587,0.1271,0.9993,0.7279,0.3581,0.2793,0.9415,0.9979,0.5790,0.0073,0.2376,0.0427


Granger Causality Test: sentiment indices vs. SP_BDM_exlarge


,1.gt_bitcoin,10.news_senti,11.fear&greed,12.tweets_pos,13.tweets_neg,14.twitter_btccrash_pos,15.twitter_btccrash_neg,2.gt_Buy_Bitcoin,3.gt_Sell_Bitcoin,4.gt_ethereum,5.gt_Ukraine_war,6.gt_covid,7.s&p_twitter,8.CBDC_uncertainty,9.CBDC_atten
1,0.7397,0.9708,0.0,0.1837,0.1542,0.9479,0.2631,0.6541,0.9523,0.7323,0.8208,0.5808,0.1120,0.5701,0.6617
2,0.8640,0.9933,0.0,0.3406,0.1746,0.9139,0.4701,0.4748,0.1174,0.8653,0.9436,0.7144,0.3041,0.1480,0.0038
3,0.8326,0.9971,0.0,0.3376,0.3258,0.9730,0.6369,0.5049,0.2320,0.9696,0.9871,0.7289,0.0027,0.0902,0.0099
4,0.9207,0.9994,0.0,0.3441,0.4721,0.9924,0.6953,0.2329,0.2442,0.9713,0.9940,0.0882,0.0005,0.1345,0.0182
5,0.9465,0.9999,0.0,0.4387,0.3442,0.9984,0.5245,0.3479,0.3449,0.9147,0.9978,0.1405,0.0011,0.1492,0.0356
